# Распознавание текста

## CRNN+CTC loss baseline

В данном ноутбуке представлен baseline модели распознавания текста с помощью CRNN модели и CTC loss. 

# 0. Установка и подгрузука библиотек

In [ ]:
!pip install numpy==1.20.3
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install opencv-python==4.5.2.52
!pip install matplotlib==3.4.2
!pip install -q albumentations==0.4.6
!pip install -q hwb

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

from hwb import HandWrittenBlot
from albumentations import Rotate
import numpy as np
import cv2
import os
import json
from matplotlib import pyplot as plt
import albumentations as A 
from albumentations.pytorch import ToTensorV2
from google.colab import drive
drive.mount('/content/drive')
from tqdm.notebook import tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Разделим трейн датасет на обучающую и валидационную подвыборки


In [ ]:
import zipfile
!mkdir data
!mkdir data/final_data
!mkdir data/final_data/train
with zipfile.ZipFile('/content/drive/MyDrive/Olimpiads/NTO_AI/data/train.zip') as zf: 
    zf.extractall('data/final_data')  # data/final_data/train/

In [ ]:
with open('data/final_data/train/labels.json') as f:
    train_data = json.load(f)

train_data = [(k, v) for k, v in train_data.items()]
print('train len', len(train_data))

split_coef = 0.85
train_len = int(len(train_data)*split_coef)

train_data_splitted = train_data[:train_len]
val_data_splitted = train_data[train_len:]

print('train len after split', len(train_data_splitted))
print('val len after split', len(val_data_splitted))


with open('data/final_data/train/train_labels_splitted.json', 'w') as f:
    json.dump(dict(train_data_splitted), f)
    
with open('data/final_data/train/val_labels_splitted.json', 'w') as f:
    json.dump(dict(val_data_splitted), f)

train len 66599
train len after split 56609
val len after split 9990


## 2. Зададим параметры обучения

Здесь мы можем поправить конфиги обучения - задать размер батча, количество эпох, размер входных изображений, а также установить пути к датасетам.

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


config_json = {
    "alphabet": " I!\"#$%&'()*+,-./0123456789:;<=>?@[\\]^_`{|}~«»ЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№",
    "save_dir": "/content/drive/MyDrive/Olimpiads/NTO_AI/data",
    "num_epochs": 500,
    "image": {
        "width": 512, 
        "height": 80  
    },
    "train": {
        "root_path": "data/final_data/train/images/",
        "json_path": "data/final_data/train/train_labels_splitted.json",
        "batch_size": 64 
    },
    "val": {
        "root_path": "data/final_data/train/images/",
        "json_path": "data/final_data/train/val_labels_splitted.json",
        "batch_size": 128 
    }
}

## 3. Теперь определим класс датасета (torch.utils.data.Dataset) и другие вспомогательные функции

In [ ]:
# функция которая помогает объединять картинки и таргет-текст в батч
def collate_fn(batch):
    images, texts, enc_texts = zip(*batch)
    images = torch.stack(images, 0)
    text_lens = torch.LongTensor([len(text) for text in texts])
    enc_pad_texts = pad_sequence(enc_texts, batch_first=True, padding_value=0)
    return images, texts, enc_pad_texts, text_lens


def get_data_loader(
    transforms, json_path, root_path, tokenizer, batch_size, drop_last
):
    dataset = OCRDataset(json_path, root_path, tokenizer, transforms)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        collate_fn=collate_fn,
        batch_size=batch_size,
        num_workers=8,
    )
    return data_loader


class OCRDataset(Dataset):
    def __init__(self, json_path, root_path, tokenizer, transform=None):
        super().__init__()
        self.transform = transform
        with open(json_path, 'r') as f:
            data = json.load(f)
        self.data_len = len(data)

        self.img_paths = []
        self.texts = []
        for img_name, text in data.items():
            self.img_paths.append(os.path.join(root_path, img_name))
            self.texts.append(text)
        self.enc_texts = tokenizer.encode(self.texts)

    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        text = self.texts[idx]
        enc_text = torch.LongTensor(self.enc_texts[idx])
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image) 
        return image, text, enc_text


class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## 4. Здесь определен Токенайзер - вспопогательный класс, который преобразует текст в числа

Разметка-текст с картинок преобразуется в числовое представление, на которых модель может учиться. Также может преобразовывать числовое предсказание модели обратно в текст.

In [ ]:
OOV_TOKEN = '<OOV>'
CTC_BLANK = '<BLANK>'


def get_char_map(alphabet):
    """Make from string alphabet character2int dict.
    Add BLANK char fro CTC loss and OOV char for out of vocabulary symbols."""
    char_map = {value: idx + 2 for (idx, value) in enumerate(alphabet)}
    char_map[CTC_BLANK] = 0
    char_map[OOV_TOKEN] = 1
    return char_map


class Tokenizer:
    """Class for encoding and decoding string word to sequence of int
    (and vice versa) using alphabet."""

    def __init__(self, alphabet):
        self.char_map = get_char_map(alphabet)
        self.rev_char_map = {val: key for key, val in self.char_map.items()}

    def encode(self, word_list):
        """Returns a list of encoded words (int)."""
        enc_words = []
        for word in word_list:
            enc_words.append(
                [self.char_map[char] if char in self.char_map
                 else self.char_map[OOV_TOKEN]
                 for char in word]
            )
        return enc_words

    def get_num_chars(self):
        return len(self.char_map)

    def decode(self, enc_word_list):
        """Returns a list of words (str) after removing blanks and collapsing
        repeating characters. Also skip out of vocabulary token."""
        dec_words = []
        for word in enc_word_list:
            word_chars = ''
            for idx, char_enc in enumerate(word):
                # skip if blank symbol, oov token or repeated characters
                if (
                    char_enc != self.char_map[OOV_TOKEN]
                    and char_enc != self.char_map[CTC_BLANK]
                    # idx > 0 to avoid selecting [-1] item
                    and not (idx > 0 and char_enc == word[idx - 1])
                ):
                    word_chars += self.rev_char_map[char_enc]
            dec_words.append(word_chars)
        return dec_words

## 5. Accuracy в качестве метрики

Accuracy измеряет долю предсказанных строк текста, которые полностью совпадают с таргет текстом.

In [ ]:
def get_accuracy(y_true, y_pred):
    scores = []
    for true, pred in zip(y_true, y_pred):
        scores.append(true == pred)
    avg_score = np.mean(scores)
    return avg_score

## 6. Аугментации

Здесь мы задаем базовые аугментации для модели

In [ ]:
blot_params = {
    'incline': (-10, 10),
    'intensivity': (0.2, 0.6),
    'transparency': (0.0, 0.35),
    'count': (0, 2)
    }

rect_config = {
        'x': (None, None),
        'y': (None, None),
        'h': (None, 0),
        'w': (None, 0)
        }      

class HandWrittenBlots:
    def __call__(self, img):
        y, x, z = img.shape
        rect_config['h'], rect_config['w'] = (None, y//2), (None, x//4)
        blots = HandWrittenBlot(rect_config, blot_params)
        img = blots(img)
        return img


class Normalize:
    def __call__(self, img):
        img = img.astype(np.float32) / 255
        return img


class ToTensor:
    def __call__(self, arr):
        arr = torch.from_numpy(arr)
        return arr


class MoveChannels:
    """Move the channel axis to the zero position as required in pytorch."""

    def __init__(self, to_channels_first=True):
        self.to_channels_first = to_channels_first

    def __call__(self, image):
        if self.to_channels_first:
            return np.moveaxis(image, -1, 0)
        else:
            return np.moveaxis(image, 0, -1)

class albums:
    def __call__(self, image):
      transform = A.Compose([                  
          A.RandomBrightness(limit=0.3, p=.8),
          A.CLAHE(p=.3),
          A.ToGray(p=.3),
          A.GridDistortion(p=.3, border_mode=cv2.BORDER_CONSTANT, num_steps=5, distort_limit=0.3),
          A.GaussianBlur(blur_limit=(1, 3), always_apply=False, p=0.35),
          Rotate(limit=5, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, value=None, mask_value=None, always_apply=False, p=0.5) 
        ])
      image = transform(image=image)['image']
      return image

class ImageResize:
    def __call__(self, img): 
        y, x, z = img.shape
        difference = 64 / y 
        y = int(y * difference)
        x = int(x * difference)
        if x > 512: 
            x = 512
        img = cv2.resize(img, (x,y), interpolation=cv2.INTER_LINEAR)
        f = np.zeros((64,512,3),np.uint8)
        f[0:y,0:x] = img
        return f

def get_train_transforms(height, width):
    transforms = torchvision.transforms.Compose([
        #HandWrittenBlots(),                                                        
        albums(),
        ImageResize(),
        MoveChannels(to_channels_first=True),
        Normalize(),
        ToTensor(),
      ])
    return transforms


def get_val_transforms(height, width):
    transforms = torchvision.transforms.Compose([
        ImageResize(),
        MoveChannels(to_channels_first=True),
        Normalize(),
        ToTensor()
    ])
    return transforms

'\ntorchvision.transforms.RandomRotation(degrees=(0, 15))\ntorchvision.transforms.ColorJitter(brightness=0.2, contrast=0.15, saturation=0, hue=0),   \ntorchvision.transforms.RandomGrayscale(p=0.15),\ntorchvision.transforms.RandomRotation(degrees=(0, 15))\n'

## 7. Здесь определяем саму модель - CRNN

Подробнее об архитектуре можно почитать в статье https://arxiv.org/abs/1507.05717

In [ ]:
def get_resnet34_backbone(pretrained=True):
    m = torchvision.models.resnet34(pretrained=True) 
    input_conv = nn.Conv2d(3, 64, 7, 1, 3) 
    blocks = [input_conv, m.bn1, m.relu,
              m.maxpool, m.layer1, m.layer2, m.layer3] 
    return nn.Sequential(*blocks)


class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            dropout=dropout, batch_first=True, bidirectional=True)

    def forward(self, x):
        out, _ = self.lstm(x)
        return out


class CRNN(nn.Module):
    def __init__(
        self, number_class_symbols, time_feature_count=256, lstm_hidden=512, 
        lstm_len=4, 
    ):
        super().__init__()
        self.feature_extractor = get_resnet34_backbone(pretrained=True)
        self.avg_pool = nn.AdaptiveAvgPool2d(
            (time_feature_count, time_feature_count))
        self.bilstm = BiLSTM(time_feature_count, lstm_hidden, lstm_len)
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, time_feature_count),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(time_feature_count, number_class_symbols)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        b, c, h, w = x.size()
        x = x.view(b, c * h, w)
        x = self.avg_pool(x)
        x = x.transpose(1, 2)
        x = self.bilstm(x)
        x = self.classifier(x)
        x = nn.functional.log_softmax(x, dim=2).permute(1, 0, 2)
        return x

## 8. Переходим к самому скрипту обучения - циклы трейна и валидации

In [ ]:
import editdistance
def get_metrics(true_texts, pred_texts):
    numCharErr = 0
    numCharTotal = 0
    for i in range(len(pred_texts)):
        pred = pred_texts[i]
        true = true_texts[i]
    
        dist = editdistance.eval(pred, true)
        numCharErr += dist
        numCharTotal += len(true)
        
    charErrorRate = numCharErr / numCharTotal
    return charErrorRate 

In [ ]:
def val_loop(data_loader, model, tokenizer, device):
    acc_avg = AverageMeter()
    cer_avg = AverageMeter()
    for images, texts, _, _ in data_loader:
        batch_size = len(texts)
        text_preds = predict(images, model, tokenizer, device)
        acc_avg.update(get_accuracy(texts, text_preds), batch_size)
        cer_avg.update(get_metrics(texts, text_preds))
    print(f'CER: {cer_avg.avg:.4f},', f' acc: {acc_avg.avg:.4f}')
    return acc_avg.avg


def train_loop(data_loader, model, criterion, optimizer, epoch):
    loss_avg = AverageMeter()
    model.train()
    for images, texts, enc_pad_texts, text_lens in tqdm(data_loader, total=len(data_loader)):
        model.zero_grad()
        images = images.to(DEVICE)
        batch_size = len(texts)
        output = model(images)
        output_lenghts = torch.full(
            size=(output.size(1),),
            fill_value=output.size(0),
            dtype=torch.long
        )
        loss = criterion(output, enc_pad_texts, output_lenghts, text_lens)
        loss_avg.update(loss.item(), batch_size)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    print(f'\nEpoch {epoch}, Loss: {loss_avg.avg:.5f}, LR: {lr:.7f}')
    return loss_avg.avg


def predict(images, model, tokenizer, device):
    model.eval()
    images = images.to(device)
    with torch.no_grad():
        output = model(images)
    pred = torch.argmax(output.detach().cpu(), -1).permute(1, 0).numpy()
    text_preds = tokenizer.decode(pred)
    return text_preds


def get_loaders(tokenizer, config):
    train_transforms = get_train_transforms(
        height=config['image']['height'],
        width=config['image']['width']
    )
    train_loader = get_data_loader(
        json_path=config['train']['json_path'],
        root_path=config['train']['root_path'],
        transforms=train_transforms,
        tokenizer=tokenizer,
        batch_size=config['train']['batch_size'],
        drop_last=True
    )
    val_transforms = get_val_transforms(
        height=config['image']['height'],
        width=config['image']['width']
    )
    val_loader = get_data_loader(
        transforms=val_transforms,
        json_path=config['val']['json_path'],
        root_path=config['val']['root_path'],
        tokenizer=tokenizer,
        batch_size=config['val']['batch_size'],
        drop_last=False
    )
    return train_loader, val_loader


def train(config):
    tokenizer = Tokenizer(config['alphabet'])
    os.makedirs(config['save_dir'], exist_ok=True)
    train_loader, val_loader = get_loaders(tokenizer, config)

    model = CRNN(number_class_symbols=tokenizer.get_num_chars()) 
    model.to(DEVICE)
    # model.load_state_dict(torch.load('/content/drive/MyDrive/Olimpiads/NTO_AI/data/model_final_2.ckpt', map_location=DEVICE))

    criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.3, patience=2) 

    best_acc = -np.inf
    acc_avg = val_loop(val_loader, model, tokenizer, DEVICE)
    for epoch in range(config['num_epochs']):
        loss_avg = train_loop(train_loader, model, criterion, optimizer, epoch)
        acc_avg = val_loop(val_loader, model, tokenizer, DEVICE)
        scheduler.step(loss_avg)
        if acc_avg > best_acc:
            best_acc = acc_avg
            model_save_path = '/content/drive/MyDrive/Olimpiads/NTO_AI/data/model_final_2.ckpt' 
            torch.save(model.state_dict(), model_save_path)
            print('Model weights saved')

## 9. Запускаем обучение!

In [ ]:
train(config_json)